# Доля аудитории каналов за день и в прайм-тайм
Пример расчета доли телеканалов по нескольким ЦА за полный день и в прайм-тайм

## Описание задачи и условий расчета
- Период: 03.10.2022 - 09.10.2022
- Временной интервал: 05:00-29:00 и 19:00-23:00
- ЦА: Все 4+ Россия 100+, Все 18-54 Россия 100+
- Место просмотра: Все места (дом+дача)
- Каналы: все каналы проекта TVI									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['100+', 'возраст'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-10-03', '2022-10-09')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группы ЦА и временных интервалов

In [ ]:
# Формируем необходимые группы ЦА 
targets = {
    '4+ Россия 100+':'cube100Plus100Minus = 1',
    '18-54 Россия 100+':'age >= 18 AND age <= 54 AND cube100Plus100Minus = 1'
}

# Формируем необходимые группы временных интервалов 
timebands = {
    'total day':'timeBand1 >= 50000 AND timeBand1 < 290000',
    '19:00 - 23:00':'timeBand1 >= 190000 AND timeBand1 < 230000'
}

## Расчет задания

In [ ]:
# Посчитаем комбинации ЦА и временных интервалов в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы формируем задания и отправляем на расчет
for target, tar_log in targets.items():
    for timeband, tb_log in timebands.items():    
        # Формируем задание на расчет
        project_name = f"{target}&{timeband}" 
        basedemo_filter = tar_log
        time_filter = tb_log

        # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
        task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name    
        tsk['task'] = mtask.send_timeband_task(task_json)
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

In [ ]:
# Разделяем колонку с именем проекта на ЦА и временной интервал
df[['TA', 'Timeband']] = df['prj_name'].str.split('&', 1, expand=True)

# Удаляем исходную колонку
df = df.drop(columns=['prj_name'])

# Разворачиваем таблицу
df_res = pd.pivot_table(df, values=['Share'],
                        index=['tvNetName'], 
                        columns=['Timeband','TA'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('07_share_totalday_primetime')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)